In [ ]:
!pip install tensorflow_addons
!pip install nlpaug
!pip install sentencepiece
!pip install transformers
!pip install torch 
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import tensorflow_addons as tfa
import keras
import torchtext

from sklearn.preprocessing import MultiLabelBinarizer
seed = 2000
np.random.seed(seed)
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Embedding
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras

import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action
import sentencepiece
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from torch.nn import BCEWithLogitsLoss
import torch
from transformers import AlbertConfig, AlbertTokenizer, TFAlbertForSequenceClassification
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, AdamW
from tensorflow_addons.metrics import HammingLoss
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import hamming_loss

In [ ]:
train_sub2_df = pd.read_csv('/content/train_subtask_2.csv')

print(train_sub2_df.shape)
train_sub2_df.head()

dev_sub2_df = pd.read_csv('/content/dev_subtask_2.csv')

print(dev_sub2_df.shape)
dev_sub2_df.head()

test_sub2_df = pd.read_csv('/content/test_subtask_2.csv')
print(test_sub2_df.shape)
test_sub2_df.head()


(433, 10)
(83, 10)
(54, 10)


,Unnamed: 0,id,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,0,3121,The Pope Says 'Humanity Must Repent For Abusin...,We humans “must repent and modify our lifestyl...,the pope say humanity must repent for abuse mo...,"[(the, 'DT'), (pope, 'NN'), (say, 'VBP'), (hum...",[],we human must repent and modify our lifestyle ...,"[(we, 'PRP'), (human, 'NN'), (must, 'MD'), (re...","[(thursday, 'DATE', 391), (the world day, 'DAT..."
1,1,3135,Russian allies are unknowingly working against...,The Secretary of State for Defence described P...,russian ally be unknowingly work against they ...,"[(russian, 'JJ'), (ally, 'NN'), (be, 'VB'), (u...","[(russian, 'NORP', 381)]",the secretary of state for defence describe pu...,"[(the, 'DT'), (secretary, 'NNP'), (of, 'IN'), ...","[(putin, 'PERSON', 380), (russian, 'NORP', 381..."
2,2,3134,"I know this week has been disruptive, says LIZ...",The United Kingdom is the greatest country on ...,know this week have be disruptive say liz truss,"[(know, 'VB'), (this, 'DT'), (week, 'NN'), (ha...","[(this week, 'DATE', 391)]",the united kingdom be the great country on ear...,"[(the, 'DT'), (united, 'NNP'), (kingdom, 'NNP'...","[(the united kingdom, 'GPE', 384), (70 year, '..."
3,3,3120,Vladimir Putin facing Ukraine 'humiliation' as...,"VLADIMIR PUTIN is facing ""humiliation"" in Ukra...",vladimir putin face ukraine humiliation as rus...,"[(vladimir, 'NNP'), (putin, 'NNP'), (face, 'NN...","[(vladimir putin, 'PERSON', 380), (ukraine, 'G...",vladimir putin be face humiliation in ukraine ...,"[(vladimir, 'NNP'), (putin, 'NNP'), (be, 'VB')...","[(vladimir putin, 'PERSON', 380), (ukraine, 'G..."
4,4,3136,What Are the Odds Putin is Bluffing About Usin...,Eurointelligence founder Wolfgang Münchau has ...,what be the odd putin be bluff about use nucle...,"[(what, 'WP'), (be, 'VB'), (the, 'DT'), (odd, ...",[],eurointelligence founder wolfgang münchau have...,"[(eurointelligence, 'NN'), (founder, 'NN'), (w...","[(wolfgang münchau, 'PERSON', 380), (putin, 'P..."


In [ ]:
train_sub2_df["genre"] = train_sub2_df["genre"].str.split(',')
dev_sub2_df["genre"] = dev_sub2_df["genre"].str.split(',')

In [ ]:
train_sub2_df["genre"]

0      [Political, Morality, Fairness_and_equality, E...
1      [Morality, Security_and_defense, Policy_prescr...
2      [Political, Crime_and_punishment, External_reg...
3      [Political, Crime_and_punishment, Fairness_and...
4      [Political, Morality, Fairness_and_equality, E...
                             ...                        
428    [Political, Crime_and_punishment, External_reg...
429    [Political, Morality, Crime_and_punishment, Qu...
430    [Crime_and_punishment, Political, Legality_Con...
431    [Political, Crime_and_punishment, Fairness_and...
432    [Crime_and_punishment, Political, Legality_Con...
Name: genre, Length: 433, dtype: object

In [ ]:
dev_sub2_df["genre"]

0     [Political, Fairness_and_equality, Policy_pres...
1     [Political, Capacity_and_resources, Policy_pre...
2     [Public_opinion, Policy_prescription_and_evalu...
3     [Political, External_regulation_and_reputation...
4     [Public_opinion, Political, External_regulatio...
                            ...                        
78    [Political, Crime_and_punishment, Fairness_and...
79    [Policy_prescription_and_evaluation, Political...
80    [Policy_prescription_and_evaluation, Political...
81    [Crime_and_punishment, Political, Legality_Con...
82    [Political, Morality, Crime_and_punishment, Qu...
Name: genre, Length: 83, dtype: object

In [ ]:
mlb = MultiLabelBinarizer()
multi_train_1 = pd.DataFrame(mlb.fit_transform(train_sub2_df['genre']), columns = mlb.classes_)
multi_train = mlb.fit_transform(train_sub2_df['genre'])

mlb = MultiLabelBinarizer()
multi_dev_1 = pd.DataFrame(mlb.fit_transform(dev_sub2_df['genre']), columns = mlb.classes_)
multi_dev = mlb.fit_transform(dev_sub2_df['genre'])

In [ ]:
multi_train_1

In [ ]:
multi_dev_1

In [ ]:
multi_train

In [ ]:
type(multi_train)

In [ ]:
multi_dev

In [ ]:
# aug = naw.SynonymAug(aug_src='wordnet',aug_max=3)
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", aug_max=4)

In [ ]:
x_train = train_sub2_df.preprocessed_articles + train_sub2_df.preprocessed_headlines
y_train = multi_train

x_dev = dev_sub2_df.preprocessed_articles + dev_sub2_df.preprocessed_headlines
y_dev = multi_dev

# x_train = train_sub1_df.text
# y_train = train_sub1_df.target

x_test = test_sub2_df.preprocessed_articles + test_sub2_df.preprocessed_headlines

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', num_labels=14, output_attentions=True)

In [ ]:
x_train

In [ ]:
x_dev

In [ ]:
augmented_sentences=[]
augmented_sentences_labels=[]
count = 0

for i in x_train.index:
    count+=1
    if count%10==0:
      print("Running count = ", count)
    try:
      temps=aug.augment(x_train[i], n=3)
      for sent in temps:
        augmented_sentences.append(sent)
        augmented_sentences_labels.append(y_train[i])
    except:
      continue

Running count =  10
Running count =  20
Running count =  30
Running count =  40
Running count =  50
Running count =  60
Running count =  70
Running count =  80
Running count =  90


In [ ]:
x_train_aug = pd.Series()
y_train_aug = pd.Series()
x_train_aug = x_train_aug.append(pd.Series(augmented_sentences), ignore_index=True)
y_train_aug = y_train_aug.append(pd.Series(augmented_sentences_labels), ignore_index=False)

In [ ]:
df=pd.concat([x_train_aug,y_train_aug],axis=1)

In [ ]:
df.columns =['x_train', 'y_train']

In [ ]:
df

In [ ]:
df.to_csv('aug.csv')

In [ ]:
x_train_aug.shape

In [ ]:
y_train_aug.shape

In [ ]:
type(y_train_aug)

In [ ]:
arr = np.stack(y_train_aug.to_numpy())
arr.shape
# np_arr

In [ ]:
y_train_aug_1 = arr

In [ ]:
y_train_aug_1.shape

In [ ]:
x_train = x_train_aug.to_numpy().reshape(-1)

x_dev = x_dev.to_numpy().reshape(-1)

In [ ]:
x_train.shape

In [ ]:
def roberta_encode(texts, tokenizer):
    MAX_LEN = 512
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(str(text))
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [ ]:
x_train = roberta_encode(x_train, tokenizer)
x_dev = roberta_encode(x_dev, tokenizer)
x_test = roberta_encode(x_test, tokenizer)

y_train = np.asarray(y_train_aug_1, dtype='int32')
y_dev = np.asarray(y_dev, dtype='int32')

In [ ]:
from sklearn.metrics import f1_score


In [ ]:
def build_model(n_categories):
    MAX_LEN = 512
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

    # Import RoBERTa model from HuggingFace
    albert_model = TFAlbertForSequenceClassification.from_pretrained('albert-base-v2')
    # custom_objects = {"TFRobertaForSequenceClassification": TFRobertaForSequenceClassification}
    # config = roberta_model.get_config()
    # with tf.keras.utils.custom_object_scope(custom_objects):
    #     roberta_model = TFRobertaForSequenceClassification.from_config(config)
    x =albert_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

    # Huggingface transformers have multiple outputs, embeddings are the first one,
    # so let's slice out the first position
    x = x[0]

    x = tf.keras.layers.Dropout(rate=0.1)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(n_categories, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
    model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                  loss='binary_crossentropy', 
                  metrics=[tf.keras.metrics.BinaryAccuracy(),
                           tfa.metrics.F1Score(num_classes=14,
                                               average='micro', threshold=0.07142857)])
    
    return model

In [ ]:
x_test['input_type_ids'].shape

In [ ]:
model = build_model(14)
model.summary()

In [ ]:
x_train['input_word_ids'].shape

In [ ]:
x_dev['input_word_ids'].shape

In [ ]:
y_dev.shape

In [ ]:
y_train.shape

In [ ]:
y_dev.shape

In [ ]:
metrics=[tf.keras.metrics.BinaryAccuracy(),
                           tfa.metrics.F1Score(num_classes=14,
                                               average='micro', threshold=0.07142857)]

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = '/content/Modelcheckpoints/Checkpoint-{epoch:02d}.h5'

checkpoint = ModelCheckpoint(filepath,monitor= tfa.metrics.F1Score(num_classes=14,
                                               average='micro', threshold=0.07142857), varbose=1, save_best_only=True, mode='max')

In [ ]:
model.fit(x_train, y_train, epochs=90, batch_size=8, validation_data=(x_dev, y_dev), callbacks=[checkpoint])

In [ ]:
model.save('saved_model.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.models import load_model

best_model = load_model('/content/drive/MyDrive/saved_model.h5', custom_objects={"TFRobertaForSequenceClassification": TFRobertaForSequenceClassification})


In [ ]:
y_pred = best_model.predict(x_dev)

3/3 [==============================] - 6s 970ms/step


In [ ]:
y_pred

array([[1.6335760e-05, 4.8347715e-01, 4.6628029e-03, ..., 1.8764772e-02,
        1.9747724e-01, 6.1159277e-01],
       [4.6092998e-02, 5.3930354e-01, 1.1276957e-01, ..., 2.5708574e-01,
        3.9871091e-01, 5.8334976e-01],
       [2.0025104e-04, 5.1149935e-01, 1.2280857e-02, ..., 4.5682196e-02,
        2.5235146e-01, 6.0199082e-01],
       ...,
       [1.0783260e-05, 1.9459236e-01, 5.4322364e-14, ..., 6.5992116e-03,
        1.4807999e-02, 2.3561838e-01],
       [2.0019787e-18, 9.8314214e-01, 6.5795750e-19, ..., 9.0061949e-04,
        5.4123541e-03, 1.7794815e-01],
       [1.6827967e-03, 6.5994871e-01, 5.0248988e-03, ..., 1.4663096e-01,
        3.2711351e-01, 5.9690464e-01]], dtype=float32)

In [ ]:
y_pred_final = list()
y_pred_sub = list()

y_pred_final = tf.cast(y_pred >= 0.2495, tf.int32)

In [ ]:
from sklearn.metrics import f1_score
print('Macro f1_score = {}'.format(f1_score(y_pred_final, y_dev, average='macro')))
print('Micro f1_score = {}'.format(f1_score(y_pred_final, y_dev, average='micro')))

Macro f1_score = 0.41471131490957924
Micro f1_score = 0.6487006737247354


In [ ]:
y_test_pred = model.predict(x_test)

2/2 [==============================] - 5s 812ms/step


In [ ]:
y_test_pred

array([[0.5000522 , 0.5000372 , 0.49995482, 0.49941018, 0.5000697 ,
        0.50088733, 0.50058866, 0.49663457, 0.5030702 , 0.4992948 ,
        0.5021492 , 0.4981032 , 0.50082916, 0.50245386],
       [0.5000621 , 0.5000391 , 0.4998088 , 0.49943012, 0.5000673 ,
        0.50085676, 0.5005911 , 0.49706954, 0.50284827, 0.49933183,
        0.5018904 , 0.4981499 , 0.5007856 , 0.5020589 ],
       [0.5000752 , 0.5000465 , 0.5000637 , 0.49950635, 0.50008714,
        0.5007082 , 0.50044197, 0.497145  , 0.5025048 , 0.49942595,
        0.5017948 , 0.49851894, 0.5006423 , 0.5021427 ],
       [0.5000785 , 0.50005454, 0.5000438 , 0.4994086 , 0.50009257,
        0.5008576 , 0.50054324, 0.4965933 , 0.50301665, 0.4993086 ,
        0.50215423, 0.49819562, 0.50078505, 0.50253516],
       [0.50007397, 0.5000458 , 0.49975777, 0.4993886 , 0.5000739 ,
        0.5009147 , 0.5006328 , 0.49696133, 0.5030043 , 0.49929082,
        0.50196624, 0.49802738, 0.5008309 , 0.5021091 ],
       [0.5000788 , 0.5000546 , 0.5

In [ ]:
y_test_final = list()


y_test_final = tf.cast(y_test_pred >= 0.50, tf.int32)

In [ ]:
px = pd.DataFrame(y_test_final.numpy())

In [ ]:
px

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,1,0,0,1,1,1,0,1,0,1,0,1,1
1,1,1,0,0,1,1,1,0,1,0,1,0,1,1
2,1,1,1,0,1,1,1,0,1,0,1,0,1,1
3,1,1,1,0,1,1,1,0,1,0,1,0,1,1
4,1,1,0,0,1,1,1,0,1,0,1,0,1,1
5,1,1,1,0,1,1,1,0,1,0,1,0,1,1
6,1,1,0,0,1,1,1,0,1,0,1,0,1,1
7,1,1,0,0,1,1,1,0,1,0,1,0,1,1
8,1,1,1,0,1,1,1,0,1,0,1,0,1,1
9,1,1,0,0,1,1,1,0,1,0,1,0,1,1


In [ ]:
y_test_final.to_csv('data.csv')

AttributeError: ignored